In [1]:
# need to do this before transformer imports
# import os
# os.environ['HF_HOME'] = '/workspace/cache/huggingface/'

import os
os.chdir('/workspace/FutureGPT2/src/')
from models.myopic_model import *
from data.utils import get_tokenizer
import datasets
from torch.utils.data import DataLoader
from torch import nn
from itertools import islice
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from torch import nn
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger

from tqdm import tqdm
import pandas as pd
import gc
from glob import glob
import numpy as np
import copy
import wandb
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
NAME = 'GPT2-MYOPIC-H-FROMORIG'
PROJ = 'LAISR_FUTURE_GPT2'
model_name = 'gpt2'

In [3]:
wandb.login(key='os.environ[WANDB_API_KEY]', relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/wwu/.netrc


True

In [4]:
if torch.cuda.get_device_capability()[0] >= 8:
    torch.set_float32_matmul_precision('high')

In [5]:
train = datasets.load_from_disk('/workspace/corpus/msmarco/msmarco_GPT2_64tokens_full/train').with_format('torch')
val = datasets.load_from_disk('/workspace/corpus/msmarco/msmarco_GPT2_64tokens_full/val').with_format('torch')
train_loader = DataLoader(train, batch_size=256)
val_loader = DataLoader(val, batch_size=256)
#loaders = {
#    split: DataLoader(dataset[split], batch_size=128)
#    for split in ['train', 'val', 'test']
#}

In [6]:
wandb_logger = WandbLogger(
    name=NAME,
    project=PROJ,
    log_model=False,   # Only save checkpoints locally
)

In [7]:
lr_callback = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(
    dirpath="/workspace/checkpoints",
    filename=NAME + "_{val_myopic_loss:.2f}",
    every_n_epochs=1,
    save_top_k=1,
    monitor='val_myopic_loss',
    mode='min',
)
early_stop_callback = EarlyStopping(
    monitor='val_myopic_loss',
    divergence_threshold=15,
    min_delta=0.00,
    patience=100000,
    verbose=False,
    mode='min',
)
trainer = L.Trainer(
    fast_dev_run=False,
    logger=wandb_logger,
    val_check_interval=.1,
    callbacks=[checkpoint_callback, lr_callback, early_stop_callback],
    max_epochs=1,
    enable_progress_bar=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
config = AutoConfig.from_pretrained(model_name)
orig_model = AutoModelForCausalLM.from_config(config=config)
# orig_state = torch.load('/workspace/checkpoints/GPT2-MYOPIC-NO-ORIG_val_myopic_loss=5.34.ckpt')
orig_state = torch.load('/workspace/checkpoints/GPT2-MSMARCO-COSINE_global_step=9099.0_val_loss=3.28.ckpt')
orig_model.load_state_dict(
    {'.'.join(k.split('.')[1:]): v for k, v in orig_state['state_dict'].items()} #if 'myopic_model' in k}
)

<All keys matched successfully>

In [9]:
config = AutoConfig.from_pretrained(model_name)
myopic_model = AutoModelForCausalLM.from_config(config=config)

In [10]:
model = LitMyopicModel(
    myopic_model=myopic_model,
    orig_model=orig_model,    # set to None (default) for cutgrad training [use own detached hidden state or kv]
    #orig_model=myopic_model,
    #orig_model=None,
    loss_type = 'myopic_loss',
    from_kv=False,
)
wandb_logger.watch(model.myopic_model, log='all')

wandb: Currently logged in as: wilswu. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [11]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name         | Type            | Params
-------------------------------------------------
0 | myopic_model | GPT2LMHeadModel | 124 M 
1 | orig_model   | GPT2LMHeadModel | 124 M 
-------------------------------------------------
124 M     Trainable params
124 M     Non-trainable params
248 M     Total params
995.518   Total estimated model params size (MB)


NUM TRAINING STEPS 18201


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [12]:
torch.is_grad_enabled()

True

In [13]:
myopic_model.transformer.h[0].attn.c_attn?

Signature:      myopic_model.transformer.h[0].attn.c_attn(*args, **kwargs)
Type:           Conv1D
String form:    Conv1D()
File:           ~/.local/lib/python3.10/site-packages/transformers/pytorch_utils.py
Docstring:     
1D-convolutional layer as defined by Radford et al. for OpenAI GPT (and also used in GPT-2).

Basically works like a linear layer but the weights are transposed.

Args:
    nf (`int`): The number of output features.
    nx (`int`): The number of input features.
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.